In [1]:
# 📘 Step 1: Install and Import Required Libraries
!pip install nba_api snscrape transformers vaderSentiment shap --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 13.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import requests
from nba_api.stats.endpoints import playbyplayv2, boxscoretraditionalv2, playergamelog
from nba_api.stats.static import players
from tqdm import tqdm
import datetime
import json
import os

# Sentiment Analysis
import snscrape.modules.twitter as sntwitter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Ignore warnings for clean output
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 📘 Step 2: Get Player ID and Game Logs
# 🔍 Helper: Get Player ID from Name
def get_player_id(player_name):
    nba_players = players.get_players()
    for p in nba_players:
        if p['full_name'].lower() == player_name.lower():
            return p['id']
    return None

# 🏀 Example: Get Steph Curry’s ID and Game Logs
player_name = "Stephen Curry"
player_id = get_player_id(player_name)

if player_id:
    print(f"Player ID for {player_name}: {player_id}")
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2023').get_data_frames()[0]
    print(gamelog[['GAME_DATE', 'MATCHUP', 'MIN', 'PTS', 'REB', 'AST', 'PLUS_MINUS']].head())
else:
    print("Player not found.")

Player ID for Stephen Curry: 201939
      GAME_DATE      MATCHUP  MIN  PTS  REB  AST  PLUS_MINUS
0  APR 12, 2024  GSW vs. NOP   32   33    4    5          -8
1  APR 11, 2024    GSW @ POR   36   22    7    8           7
2  APR 09, 2024    GSW @ LAL   32   23    7    8          13
3  APR 05, 2024    GSW @ DAL   35   28    6    5          -5
4  APR 04, 2024    GSW @ HOU   31   29    6    6          29


In [4]:
# 📘 Step 3: Loop through multiple players and save game logs to CSV
player_names = ["Stephen Curry", "LeBron James", "Kevin Durant", "Giannis Antetokounmpo"]
all_logs = []

for name in tqdm(player_names):
    pid = get_player_id(name)
    if pid:
        logs = playergamelog.PlayerGameLog(player_id=pid, season='2023').get_data_frames()[0]
        logs["PLAYER_NAME"] = name
        all_logs.append(logs)
    else:
        print(f"Player not found: {name}")

if all_logs:
    full_df = pd.concat(all_logs, ignore_index=True)
    full_df.to_csv("player_game_logs_2023.csv", index=False)
    print("Game logs saved to player_game_logs_2023.csv")
    display(full_df.head())


100%|██████████| 4/4 [00:01<00:00,  3.02it/s]

Game logs saved to player_game_logs_2023.csv


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PLAYER_NAME
0,22023,201939,0022301182,"APR 12, 2024",GSW vs. NOP,L,32,12,23,0.522,...,4,5,1,0,7,2,33,-8,1,Stephen Curry
1,22023,201939,0022301169,"APR 11, 2024",GSW @ POR,W,36,8,22,0.364,...,7,8,1,2,2,0,22,7,1,Stephen Curry
2,22023,201939,0022301155,"APR 09, 2024",GSW @ LAL,W,32,7,9,0.778,...,7,8,1,2,1,2,23,13,1,Stephen Curry
3,22023,201939,0022301097,"APR 05, 2024",GSW @ DAL,L,35,9,23,0.391,...,6,5,1,0,2,2,28,-5,1,Stephen Curry
4,22023,201939,0022301113,"APR 04, 2024",GSW @ HOU,W,31,9,14,0.643,...,6,6,0,0,3,2,29,29,1,Stephen Curry


Twitter Sentiment for Each Player on Game Day

In [7]:
# ✅ Fix SSL: Install certifi CA bundle and set environment variable
import os
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()
print("✅ SSL certificate fix applied.")


✅ SSL certificate fix applied.


In [9]:
# 📘 Step 4: Get Twitter Sentiment for Each Game Day
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(player_name, game_date):
    query = f'"{player_name}" since:{game_date} until:{game_date} lang:en'
    tweets = []

    try:
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
            if i >= 100:  # limit to 100 tweets
                break
            tweets.append(tweet.content)
    except Exception as e:
        print(f"⚠️ Error scraping tweets for {player_name} on {game_date}: {e}")
        return 0

    if not tweets:
        return 0

    scores = [analyzer.polarity_scores(t)['compound'] for t in tweets]
    return np.mean(scores)

# 🧪 Test sentiment
sample_date = '2023-10-24'
score = get_sentiment_score("Stephen Curry", sample_date)
print(f"Avg Sentiment Score for Stephen Curry on {sample_date}: {score}")


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2023-10-24+until%3A2023-10-24+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2023-10-24+until%3A2023-10-24+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2023-10-24+until%3A2023-10-24+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2023-10-24+until

⚠️ Error scraping tweets for Stephen Curry on 2023-10-24: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2023-10-24+until%3A2023-10-24+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
Avg Sentiment Score for Stephen Curry on 2023-10-24: 0


Merge Twitter Sentiment into Player Game Logs

In [10]:
# 📘 Step 5: Merge Twitter Sentiment into Game Logs

# Load previous game logs
df = pd.read_csv("player_game_logs_2023.csv")
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Add a new column for sentiment
sentiments = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    player = row["PLAYER_NAME"]
    date = row["GAME_DATE"].strftime("%Y-%m-%d")
    score = get_sentiment_score(player, date)
    sentiments.append(score)

df["TWITTER_SENTIMENT"] = sentiments

# Save with sentiment
df.to_csv("player_game_logs_with_sentiment.csv", index=False)
print("✅ Saved with Twitter sentiment.")
display(df[['PLAYER_NAME', 'GAME_DATE', 'PTS', 'TWITTER_SENTIMENT']].head())


  0%|          | 0/293 [00:00<?, ?it/s]ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22St

⚠️ Error scraping tweets for Stephen Curry on 2024-04-12: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until

⚠️ Error scraping tweets for Stephen Curry on 2024-04-11: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-09+until%3A2024-04-09+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-09+until%3A2024-04-09+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-09+until%3A2024-04-09+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-09+until

...

ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokou

⚠️ Error scraping tweets for Giannis Antetokounmpo on 2023-10-29: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokou

⚠️ Error scraping tweets for Giannis Antetokounmpo on 2023-10-26: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
✅ Saved with Twitter sentiment.


,PLAYER_NAME,GAME_DATE,PTS,TWITTER_SENTIMENT
0,Stephen Curry,2024-04-12,33,0
1,Stephen Curry,2024-04-11,22,0
2,Stephen Curry,2024-04-09,23,0
3,Stephen Curry,2024-04-05,28,0
4,Stephen Curry,2024-04-04,29,0


Load Data & Define Fatigue Function

In [11]:
# 📁 Load game logs
df = pd.read_csv("player_game_logs_2023.csv")
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# 🧠 Fatigue = Rolling 3-game sum of minutes before current game
def compute_fatigue(player_df):
    player_df = player_df.sort_values('GAME_DATE')
    player_df['FATIGUE'] = player_df['MIN'].shift(1).rolling(window=3).sum().fillna(0)
    return player_df

# 🧪 Apply fatigue calculation for each player
df = df.groupby('PLAYER_NAME').apply(compute_fatigue).reset_index(drop=True)


Compute Sentiment for Each Player on Each Game Date

In [12]:
from tqdm.notebook import tqdm

# 📘 Add sentiment score for each player/game date
sentiment_scores = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    player = row['PLAYER_NAME']
    date = row['GAME_DATE'].strftime('%Y-%m-%d')
    score = get_sentiment_score(player, date)
    sentiment_scores.append(score)

df['SENTIMENT'] = sentiment_scores


  0%|          | 0/293 [00:00<?, ?it/s]

ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokou

⚠️ Error scraping tweets for Giannis Antetokounmpo on 2023-10-26: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-26+until%3A2023-10-26+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Giannis+Antetokou

⚠️ Error scraping tweets for Giannis Antetokounmpo on 2023-10-29: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Giannis+Antetokounmpo%22+since%3A2023-10-29+until%3A2023-10-29+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


...

⚠️ Error scraping tweets for Stephen Curry on 2024-04-09: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-09+until%3A2024-04-09+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until

⚠️ Error scraping tweets for Stephen Curry on 2024-04-11: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-11+until%3A2024-04-11+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


ERROR:snscrape.base:Error retrieving https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))"))
CRITICAL:snscrape.base:4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.
CRITICAL:snscrape.base:Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until

⚠️ Error scraping tweets for Stephen Curry on 2024-04-12: 4 requests to https://twitter.com/search?f=live&lang=en&q=%22Stephen+Curry%22+since%3A2024-04-12+until%3A2024-04-12+lang%3Aen&src=spelling_expansion_revert_click failed, giving up.


In [13]:
# 🧪 Final feature preview
df[['PLAYER_NAME', 'GAME_DATE', 'MIN', 'PTS', 'REB', 'AST', 'PLUS_MINUS', 'FATIGUE', 'SENTIMENT']].head()


,PLAYER_NAME,GAME_DATE,MIN,PTS,REB,AST,PLUS_MINUS,FATIGUE,SENTIMENT
0,Giannis Antetokounmpo,2023-10-26,35,23,12,3,-13,0.0,0
1,Giannis Antetokounmpo,2023-10-29,29,26,11,3,-11,0.0,0
2,Giannis Antetokounmpo,2023-10-30,32,33,7,2,18,0.0,0
3,Giannis Antetokounmpo,2023-11-01,28,16,4,2,-10,96.0,0
4,Giannis Antetokounmpo,2023-11-03,36,22,8,6,13,89.0,0


Add Match Margin and Clutch Labels

In [14]:
# 📌 Extract opponent score diff from matchup info (e.g., "GSW vs. LAL")
def extract_margin(matchup_str):
    # crude margin: assume close games if "vs." (home game)
    return 5  # You can adjust with real score margin later using box scores

# 💡 Label clutch performance based on PLUS_MINUS and margin
df['MARGIN'] = df['MATCHUP'].apply(extract_margin)
df['CLUTCH'] = ((df['PLUS_MINUS'] >= 5) & (df['MARGIN'] <= 5)).astype(int)

# ✅ Preview labels
df[['PLAYER_NAME', 'GAME_DATE', 'PLUS_MINUS', 'MARGIN', 'CLUTCH']].head()


,PLAYER_NAME,GAME_DATE,PLUS_MINUS,MARGIN,CLUTCH
0,Giannis Antetokounmpo,2023-10-26,-13,5,0
1,Giannis Antetokounmpo,2023-10-29,-11,5,0
2,Giannis Antetokounmpo,2023-10-30,18,5,1
3,Giannis Antetokounmpo,2023-11-01,-10,5,0
4,Giannis Antetokounmpo,2023-11-03,13,5,1


Prepare Dataset for LSTM (Sequences of Stats)

In [45]:
from sklearn.preprocessing import MinMaxScaler

sequence_length = 5
features = ['PTS', 'REB', 'AST', 'MIN', 'FATIGUE', 'SENTIMENT']
sequences = []
labels = []

# Scale features
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Generate sequences for each player
for player in df['PLAYER_NAME'].unique():
    player_df = df[df['PLAYER_NAME'] == player].sort_values('GAME_DATE')
    for i in range(sequence_length, len(player_df)):
        seq = player_df.iloc[i-sequence_length:i][features].values
        label = player_df.iloc[i]['CLUTCH']
        sequences.append(seq)
        labels.append(label)

X = np.array(sequences)
y = np.array(labels)

print(f"✅ Sequence shape: {X.shape}, Labels shape: {y.shape}")


✅ Sequence shape: (273, 5, 6), Labels shape: (273,)


Train LSTM Model for Clutch Prediction

In [55]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ⛓️ Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🧠 Define LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

# 📋 Summary
model.summary()
model.save('lstm_model.h5')


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 64)             │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,289 (79.25 KB)

 Trainable params: 20,289 (79.25 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# 🚀 Train LSTM model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10


LookupError: gradient registry has no entry for: shap_DivNoNan

In [57]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 🔍 Evaluate on test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

print("✅ Accuracy:", accuracy_score(y_test, y_pred_classes))
print("\n🧾 Classification Report:\n", classification_report(y_test, y_pred_classes))
print("\n🔀 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_classes))


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
✅ Accuracy: 0.4909090909090909

🧾 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.49      1.00      0.66        27

    accuracy                           0.49        55
   macro avg       0.25      0.50      0.33        55
weighted avg       0.24      0.49      0.32        55


🔀 Confusion Matrix:
 [[ 0 28]
 [ 0 27]]


Prepare SHAP Explainer

In [58]:
import shap
import tensorflow as tf

# Load background samples for reference
background = X_train[np.random.choice(X_train.shape[0], 50, replace=False)]

# Create SHAP explainer for LSTM model
explainer = shap.DeepExplainer(model, background)


In [53]:
# Explain predictions on the test set (first 10 samples)
shap_values = explainer.shap_values(X_test[:10])


StagingError: in user code:

    File "/usr/local/lib/python3.11/dist-packages/shap/explainers/_deep/deep_tf.py", line 265, in grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)

    LookupError: gradient registry has no entry for: shap_CudnnRNNV3


In [32]:
import shap
import tensorflow as tf

# ⚙️ Use KernelExplainer for LSTM (since SHAP can't directly interpret LSTM)
explainer = shap.Explainer(model, X_train[:100])  # Limit to a small set for speed
shap_values = explainer(X_test[:10])  # Interpret first 10 test examples


TypeError: 'numpy.ndarray' object is not callable

In [59]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [60]:
%%writefile app.py
import streamlit as st
import numpy as np
from keras.models import load_model
from utils.preprocess import get_sentiment_score, preprocess_input

# Load model
model = load_model('model/lstm_model.h5')

st.title("🧠 NBA Clutch Performance Predictor")
st.markdown("Predict if a player will perform clutch based on sentiment + stats.")

# User input
player = st.text_input("Enter Player Name", "Stephen Curry")
date = st.date_input("Select Game Date")
run = st.button("Predict")

if run:
    date_str = date.strftime('%Y-%m-%d')

    # Get sentiment score
    sentiment = get_sentiment_score(player, date_str)

    # Preprocess input for model
    X_input = preprocess_input(sentiment)  # shape must match LSTM input shape

    # Predict
    pred = model.predict(X_input)[0][0]
    label = "🔥 Clutch Performance" if pred > 0.5 else "😐 Not Clutch"

    st.metric("Sentiment Score", round(sentiment, 3))
    st.metric("Prediction Score", round(pred, 3))
    st.success(f"Prediction: {label}")


Writing app.py


In [61]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.161.53:8501

  Stopping...
  Stopping...


In [66]:
# Kill any existing ngrok and Streamlit processes
!pkill -f streamlit
!pkill -f ngrok

# Re-run ngrok authentication (Replace with your token)
!ngrok authtoken 2vv97TUmiiKJefidcGYrOiANXo5_3DYwdmHHiMgPcggzURTGo

# Start Streamlit again
!nohup streamlit run app.py --server.port 8501 &

# Reconnect ngrok to expose Streamlit
from pyngrok import ngrok
import time

time.sleep(5)  # Wait for Streamlit to start
public_url = ngrok.connect(8501)
print(f" New Public Link: {public_url}")

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: appending output to 'nohup.out'


ERROR:pyngrok.process:t=2025-04-18T22:31:23+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your ngrok-agent version \"2.3.41\" is too old. The minimum supported agent version for your account is \"3.7.0\". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n"



PyngrokNgrokError: The ngrok process errored on start: Your ngrok-agent version "2.3.41" is too old. The minimum supported agent version for your account is "3.7.0". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.\r\n\r\nERR_NGROK_121\r\n.